# Tutorial on manipulating SYN L2 AOD product from the cloud

In [ ]:
import datatree
import xarray as xr
from pathlib import Path
from upath import UPath
from xarray_eop.eop import open_eop_datatree

### Get AWS credentials

In [ ]:
import s3fs
import json
# Assuming credentials are in ~/.eopf/secrets.json
try:
    SECRET_PATH = Path.home() / Path(".eopf/secrets.json")
    with open(SECRET_PATH) as f:
        secrets=json.load(f)

# Or use ENV variables
except:
    import os
    secrets = {"s3input" : {
        "key": os.getenv("AWS_ACCESS_KEY_ID"),
        "secret": os.getenv("AWS_SECRET_ACCES_KEY"),
        "endpoint_url": os.getenv("AWS_ENDPOINT_URL"),
        "region_name": os.getenv("AWS_DEFAULT_REGION")
        }
    }

secrets["s3input"].pop("region_name",None)



### Browse S3 buckets and get SYNAOD product

In [ ]:
import s3fs
SAMPLE_PATH = "s3://s3-input/Products/NewFormat/"
s3 = s3fs.S3FileSystem(
    key=secrets["s3input"]["key"],
    secret=secrets["s3input"]["secret"],
    endpoint_url=secrets["s3input"]["endpoint_url"]
)
s3_path = s3.glob(SAMPLE_PATH+"S03SYNAOD*.zip")
store=f"zip::s3://{s3_path[0]}"
store

### Open the whole product with datatree

In [ ]:
# Custom open datatree
dt = open_eop_datatree(store,backend_kwargs={"storage_options": {"s3":secrets["s3input"]}})
# Standard open datatree
# dt = datatree.open_datatree(store,engine="zarr",chunks={},backend_kwargs={"storage_options": {"s3":secrets["s3input"]}})
dt

### Opening gridded measurement data

In [ ]:
ds=dt.measurements
aod550 = ds.aod_550
aod550

Underlying data is dask.array

In [ ]:
aod550.data

#### Simple raster plot

In [ ]:
aod550.plot()

#### Plot using the coordinates (lon,lat)
Note that in SYN AOD product, lat/lon are undefined when the data is missing, which is not correctly handled by matplotlib pcolormesh

In [ ]:
# Remove margins
aod550_dropna=aod550.dropna("columns",how="all")
aod550_dropna=aod550_dropna.dropna("rows",how="all")

# Fill remaining missing values
aod550_dropna["latitude"] = (["rows","columns"],aod550_dropna.latitude.bfill("columns").data)
aod550_dropna["longitude"] = (["rows","columns"],aod550_dropna.longitude.bfill("columns").data)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(14, 6))
ax = plt.axes()
aod550_dropna.plot.pcolormesh(
    ax=ax,
    x="longitude", y="latitude", add_colorbar=False
)

#### Open uncertainties

In [ ]:
ds=dt.quality
ds